In [1]:
import pandas as pd

In [2]:
df_malaga = pd.read_csv("../data/turismo_receptor_provincia_pais_AGP.csv", encoding= "latin1", sep = ";")

df_malaga.head(10)


,AÑO,MES,PROVINCIA_DESTINO,CONTINENTE_ORIGEN,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,2022,1,Málaga,América,Argentina,"592,0","7104,0","12,0"
1,2022,1,Málaga,América,Brasil,"371,0","4526,0","12,2"
2,2022,1,Málaga,América,Canadá,"1183,0","10174,0","8,6"
3,2022,1,Málaga,América,Chile,"110,0","1089,0","9,9"
4,2022,1,Málaga,América,Colombia,"624,0","6926,0","11,1"
5,2022,1,Málaga,América,Ecuador,"54,0","545,0","10,1"
6,2022,1,Málaga,América,Estados Unidos de América,"3722,0","38337,0","10,3"
7,2022,1,Málaga,América,México,"679,0","5704,0","8,4"
8,2022,1,Málaga,América,Nicaragua,"40,0","456,0","11,4"
9,2022,1,Málaga,América,Panamá,"53,0","488,0","9,2"


In [3]:
df_malaga.loc[df_malaga.duplicated()]    # no hay duplicados

,AÑO,MES,PROVINCIA_DESTINO,CONTINENTE_ORIGEN,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA


In [4]:
df_malaga.info()   
# Vemos que no hay valores nulos
# Las columnas AÑO Y MES no están en datetime, pero MES la trataremos de categórica y AÑO la borraremos porque solo estudiaremos los datos del 2024
# Las columnas numéricas TURISTAS, PERNOCTACIONES Y ESTANCIA_MEDIA no aparecen como tal, sino como tipo object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723 entries, 0 to 3722
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   AÑO                3723 non-null   int64 
 1   MES                3723 non-null   int64 
 2   PROVINCIA_DESTINO  3723 non-null   object
 3   CONTINENTE_ORIGEN  3723 non-null   object
 4   PAIS_ORIGEN        3723 non-null   object
 5   TURISTAS           3723 non-null   object
 6   PERNOCTACIONES     3723 non-null   object
 7   ESTANCIA_MEDIA     3723 non-null   object
dtypes: int64(2), object(6)
memory usage: 232.8+ KB


In [5]:
# empezamos con el tratamiento de las columnas numéricas:

df_malaga.TURISTAS = df_malaga.TURISTAS.str.replace(",",".").astype(float)
df_malaga.PERNOCTACIONES = df_malaga.PERNOCTACIONES.str.replace(",",".").astype(float)
df_malaga.ESTANCIA_MEDIA = df_malaga.ESTANCIA_MEDIA.str.replace(",",".").astype(float)

In [6]:
es_enero = df_malaga.MES == 1
es_febrero = df_malaga.MES == 2
es_marzo = df_malaga.MES == 3
es_noviembre = df_malaga.MES == 11
es_diciembre = df_malaga.MES == 12

temp_baja = (es_enero | es_febrero | es_marzo | es_noviembre | es_diciembre)  

df_malaga.drop(df_malaga[temp_baja].index, inplace=True) # borramos las columnas de temporada baja que no nos interesan 

no_es_europa = df_malaga.CONTINENTE_ORIGEN != "Europa"
df_malaga.drop(df_malaga[no_es_europa].index, inplace=True) #borramos todo lo que no sea Europa

no_es_2024 = df_malaga.AÑO != 2024
df_malaga.drop(df_malaga[no_es_2024].index, inplace= True)  # y por último me quedo solo con los registros del 2024

In [7]:
# Ahora prescindimos de las columnas "PROVINCIA_DESTINO", pues es solo Málaga; de "CONTINENTE_ORIGEN", pues solo nos interesa Europa; y de "AÑO" ya que solo tenemos 2024
df_malaga.drop(columns= ["PROVINCIA_DESTINO", "AÑO", "CONTINENTE_ORIGEN"], inplace= True)
df_malaga

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
2751,4,Albania,92.0,580.0,6.3
2752,4,Alemania,46015.0,395729.0,8.6
2753,4,Andorra,148.0,1125.0,7.6
2754,4,Armenia,70.0,532.0,7.6
2755,4,Austria,9273.0,76039.0,8.2
...,...,...,...,...,...
3471,10,Rusia,2303.0,22339.0,9.7
3472,10,Serbia,609.0,3410.0,5.6
3473,10,Suecia,36711.0,271661.0,7.4
3474,10,Suiza,20426.0,140939.0,6.9


In [8]:
df_malaga.reset_index(drop=True, inplace=True)  # reset del indice después de los cambios pertinentes

In [9]:
# vamos a crear una nueva variable categórica en función de la temporada a la que pertenezca cada mes bajo el siguiente criterio:
# pre-temporada: abril y mayo
# temporada alta: junio, julio y agosto
# post-temporada: septiembre y octubre 

df_malaga["TEMPORADA"] = "Temporada Alta"
df_malaga.loc[df_malaga["MES"] < 6, "TEMPORADA"] = "Pre-Temporada"
df_malaga.loc[df_malaga["MES"] > 8, "TEMPORADA"] = "Post-Temporada" 
df_malaga

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
0,4,Albania,92.0,580.0,6.3,Pre-Temporada
1,4,Alemania,46015.0,395729.0,8.6,Pre-Temporada
2,4,Andorra,148.0,1125.0,7.6,Pre-Temporada
3,4,Armenia,70.0,532.0,7.6,Pre-Temporada
4,4,Austria,9273.0,76039.0,8.2,Pre-Temporada
...,...,...,...,...,...,...
295,10,Rusia,2303.0,22339.0,9.7,Post-Temporada
296,10,Serbia,609.0,3410.0,5.6,Post-Temporada
297,10,Suecia,36711.0,271661.0,7.4,Post-Temporada
298,10,Suiza,20426.0,140939.0,6.9,Post-Temporada


In [10]:
# Ahora separo mis datos por temporadas mediante una máscara boolerana

# TemporadamAlta:

temp_alta = df_malaga.TEMPORADA == "Temporada Alta"
df_verano = df_malaga[temp_alta]

df_verano


,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
86,6,Albania,101.0,576.0,5.7,Temporada Alta
87,6,Alemania,46888.0,356349.0,7.6,Temporada Alta
88,6,Andorra,175.0,1505.0,8.6,Temporada Alta
89,6,Armenia,97.0,601.0,6.2,Temporada Alta
90,6,Austria,9772.0,80130.0,8.2,Temporada Alta
...,...,...,...,...,...,...
210,8,Rusia,2831.0,32557.0,11.5,Temporada Alta
211,8,Serbia,698.0,4816.0,6.9,Temporada Alta
212,8,Suecia,35849.0,311886.0,8.7,Temporada Alta
213,8,Suiza,16851.0,134808.0,8.0,Temporada Alta


In [11]:
# Pre-temporada

pre_temp = df_malaga.TEMPORADA == "Pre-Temporada"
df_pre = df_malaga[pre_temp]

df_pre

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
0,4,Albania,92.0,580.0,6.3,Pre-Temporada
1,4,Alemania,46015.0,395729.0,8.6,Pre-Temporada
2,4,Andorra,148.0,1125.0,7.6,Pre-Temporada
3,4,Armenia,70.0,532.0,7.6,Pre-Temporada
4,4,Austria,9273.0,76039.0,8.2,Pre-Temporada
...,...,...,...,...,...,...
81,5,Rusia,2311.0,21955.0,9.5,Pre-Temporada
82,5,Serbia,665.0,4123.0,6.2,Pre-Temporada
83,5,Suecia,31109.0,211541.0,6.8,Pre-Temporada
84,5,Suiza,13539.0,85296.0,6.3,Pre-Temporada


In [12]:
# y Post-temporada:

post_temp = df_malaga.TEMPORADA == "Post-Temporada"
df_post = df_malaga[post_temp]

df_post

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
215,9,Albania,145.0,769.0,5.3,Post-Temporada
216,9,Alemania,51150.0,424545.0,8.3,Post-Temporada
217,9,Andorra,215.0,2193.0,10.2,Post-Temporada
218,9,Armenia,121.0,895.0,7.4,Post-Temporada
219,9,Austria,10410.0,86403.0,8.3,Post-Temporada
...,...,...,...,...,...,...
295,10,Rusia,2303.0,22339.0,9.7,Post-Temporada
296,10,Serbia,609.0,3410.0,5.6,Post-Temporada
297,10,Suecia,36711.0,271661.0,7.4,Post-Temporada
298,10,Suiza,20426.0,140939.0,6.9,Post-Temporada


In [13]:
# De cada uno de los 3 datasets anteriores, me voy a quedar solo con el top 15 de los países con mayor num de turistas 

top15_verano = df_verano.nlargest(n=15, columns= "TURISTAS")
top15_verano



,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
206,8,Reino Unido,228875.0,1938169.0,7.40,Temporada Alta
163,7,Reino Unido,226690.0,1694595.0,6.55,Temporada Alta
120,6,Reino Unido,220636.0,1540936.0,6.25,Temporada Alta
187,8,Francia,89592.0,707777.0,7.90,Temporada Alta
203,8,Países Bajos,74741.0,560558.0,7.50,Temporada Alta
148,7,Irlanda,61319.0,539607.0,8.80,Temporada Alta
105,6,Irlanda,60493.0,489993.0,8.10,Temporada Alta
191,8,Irlanda,56634.0,498379.0,8.80,Temporada Alta
173,8,Alemania,51420.0,457638.0,8.90,Temporada Alta
144,7,Francia,50978.0,372139.0,7.30,Temporada Alta


In [14]:
top15_pre = df_pre.nlargest(n=15, columns= "TURISTAS")
top15_pre

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
77,5,Reino Unido,206335.0,1316983.0,5.55,Pre-Temporada
34,4,Reino Unido,173947.0,1143628.0,5.85,Pre-Temporada
62,5,Irlanda,60174.0,385114.0,6.40,Pre-Temporada
74,5,Países Bajos,51579.0,340421.0,6.60,Pre-Temporada
19,4,Irlanda,47048.0,319926.0,6.80,Pre-Temporada
1,4,Alemania,46015.0,395729.0,8.60,Pre-Temporada
58,5,Francia,44404.0,293066.0,6.60,Pre-Temporada
44,5,Alemania,43551.0,330988.0,7.60,Pre-Temporada
31,4,Países Bajos,41591.0,262023.0,6.30,Pre-Temporada
15,4,Francia,38841.0,275771.0,7.10,Pre-Temporada


In [15]:
top15_post = df_post.nlargest(n=15, columns= "TURISTAS")
top15_post

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
248,9,Reino Unido,224585.0,1592646.0,6.70,Post-Temporada
291,10,Reino Unido,191486.0,1356453.0,6.25,Post-Temporada
258,10,Alemania,61724.0,506137.0,8.20,Post-Temporada
234,9,Irlanda,57748.0,386912.0,6.70,Post-Temporada
245,9,Países Bajos,53326.0,325289.0,6.10,Post-Temporada
288,10,Países Bajos,52166.0,323429.0,6.20,Post-Temporada
276,10,Irlanda,51356.0,364628.0,7.10,Post-Temporada
216,9,Alemania,51150.0,424545.0,8.30,Post-Temporada
230,9,Francia,43736.0,323646.0,7.40,Post-Temporada
272,10,Francia,39391.0,271798.0,6.90,Post-Temporada


In [16]:
# ahora que tengo los países que más han visitado Málaga por temporadas, concateno los datasets:

df_top_viajeros = pd.concat([top15_pre, top15_verano, top15_post])
df_top_viajeros.PAIS_ORIGEN.value_counts() # ahora ya manejamos un número de países con el que podemos trabajar

PAIS_ORIGEN
Reino Unido     7
Irlanda         7
Países Bajos    7
Alemania        7
Francia         6
Suecia          5
Bélgica         4
Dinamarca       2
Name: count, dtype: int64

In [17]:
df_top_viajeros

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
77,5,Reino Unido,206335.0,1316983.0,5.55,Pre-Temporada
34,4,Reino Unido,173947.0,1143628.0,5.85,Pre-Temporada
62,5,Irlanda,60174.0,385114.0,6.40,Pre-Temporada
74,5,Países Bajos,51579.0,340421.0,6.60,Pre-Temporada
19,4,Irlanda,47048.0,319926.0,6.80,Pre-Temporada
1,4,Alemania,46015.0,395729.0,8.60,Pre-Temporada
58,5,Francia,44404.0,293066.0,6.60,Pre-Temporada
44,5,Alemania,43551.0,330988.0,7.60,Pre-Temporada
31,4,Países Bajos,41591.0,262023.0,6.30,Pre-Temporada
15,4,Francia,38841.0,275771.0,7.10,Pre-Temporada


In [18]:
#df_top_viajeros.reset_index(drop=True, inplace=True) # de nuevo hacemos un reset del índice para que nos salga bien ordenado, además de hacer un sort values por meses
df_top_viajeros.sort_values("MES", ascending=True, inplace=True)
df_top_viajeros

,MES,PAIS_ORIGEN,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA,TEMPORADA
34,4,Reino Unido,173947.0,1143628.0,5.85,Pre-Temporada
19,4,Irlanda,47048.0,319926.0,6.80,Pre-Temporada
1,4,Alemania,46015.0,395729.0,8.60,Pre-Temporada
11,4,Dinamarca,21590.0,170561.0,7.90,Pre-Temporada
8,4,Bélgica,25104.0,173218.0,6.90,Pre-Temporada
40,4,Suecia,31927.0,249031.0,7.80,Pre-Temporada
15,4,Francia,38841.0,275771.0,7.10,Pre-Temporada
31,4,Países Bajos,41591.0,262023.0,6.30,Pre-Temporada
44,5,Alemania,43551.0,330988.0,7.60,Pre-Temporada
58,5,Francia,44404.0,293066.0,6.60,Pre-Temporada


In [19]:
df_top_viajeros.to_csv("../data/dataset_viajeros_procesado.csv", encoding="latin1", sep= ";") # guardo mis datasets procesado en csv para acceder a él en otros notebooks
